In [1]:
import numpy as np                      # Data
import pandas as pd                     # Data 
import geopandas as gpd                 # Data
import xarray as xr                     # Data
import atlite                           # Model
import matplotlib.pyplot as plt         # Plot
from matplotlib.lines import Line2D     # Plot
from tqdm import tqdm                   # Visualise progression in loop
import yaml                             # Open yaml files

Function to identify RES droughts and store them in a single dataframe with the index being the starting date and 

In [2]:
def identify_drought(time_series: pd.Series, threshold:float):
    events_thresh = time_series < threshold
    raw_events = events_thresh.groupby(events_thresh.ne(events_thresh.shift()).cumsum()).cumsum()
    identified_events = raw_events[(raw_events.shift(1) < raw_events) & (raw_events.shift(-1) < raw_events)]
    identified_events.index = pd.Index([date - pd.Timedelta(identified_events.loc[date]-1, unit='h') for date in identified_events.index])
    return identified_events

Load CF values

In [ ]:
df_cf_pv = pd.read_csv('../Data/cf_pv_7923.csv',
                        index_col = 0,
                        parse_dates = True
)

In [ ]:
df_cf_wind = pd.read_csv('../Data/cf_wind_7923.csv',
                        index_col = 0,
                        parse_dates = True
)

Take rolling average of the values

In [ ]:
df_cf_pv_avg = df_cf_pv.rolling('D', min_periods=24, center=True).mean()
df_cf_wind_avg = df_cf_wind.rolling('D', min_periods=24, center=True).mean()

Create a dictonary to store all the DataFrame created from the function

In [ ]:
droughts_dict = {}

for column in df_cf_pv_avg:
    raw_droughts_pv = identify_drought(df_cf_pv_avg[column], 0.1)
    droughts_pv = raw_droughts_pv[raw_droughts_pv > 24]
    droughts_df_pv = pd.DataFrame(droughts_pv)
    droughts_df_pv = droughts_df_pv.set_axis(['Duration'], axis=1)
    droughts_dict[column] = droughts_df_pv

Save 